In [ ]:
from simpletransformers.ner import NERModel, NERArgs
import numpy as np
import pandas as pd
from transformers import BertModel
from torch import nn


class Bertencoder(nn.Module):
    def __init__(self, bert_dim, output_dim, num_layers, rnn_dim):
        super(Bertencoder, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        self.rnn = nn.LSTM(bert_dim, rnn_dim, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(rnn_dim*2, output_dim)
        self.loss_function = nn.CrossEntropyLoss()

    def forward(self, x, y, seg_ids, mask):
        bert_output = self.bert_model(x, token_type_ids=seg_ids, attention_mask=mask)
        lstm_output, _ = self.rnn(bert_output.last_hidden_state)
        output = self.linear(lstm_output).transpose(1, 2)
        loss = self.loss_function(output, y)
        return loss

    def test(self, x, y, seg_ids, mask):
        bert_output = self.bert_model(x, token_type_ids=seg_ids, attention_mask=mask)
        lstm_output, _ = self.rnn(bert_output.last_hidden_state)
        output = self.linear(lstm_output).transpose(1, 2)
        loss = self.loss_function(output, y)
        predict = self.predict_label(output.transpose(1, 2), mask)
        return loss, predict

    def predict(self, x, seg_ids, mask):
        bert_output = self.bert_model(x, token_type_ids=seg_ids, attention_mask=mask)
        lstm_output, _ = self.rnn(bert_output.last_hidden_state)
        output = self.linear(lstm_output).transpose(1, 2)
        return output.transpose(1, 2).argmax(2)

    def predict_label(self, output, mask):
        """
        output: batch_size, max_sentence_len, tag_num
        mask: same label data
        batch_label: batch_size, sentence_label
        """
        pre_label = []
        pre_label_pad = output.argmax(2)
        for i in range(mask.shape[0]):
            sent_label = []
            for j in range(mask.shape[1]):
                if mask[i, j] == 1:
                    sent_label.append(pre_label_pad[i, j].item())
            sent_label = sent_label[1:len(sent_label)-1]
            pre_label.append(sent_label)

        return pre_label


In [ ]:
special_labels = {',': 'I-COMMA',
                  '.': 'I-DOT',
                  '?': 'I-QMARK',
                  '!': 'I-EMARK',
                  ':': 'I-COLON',
                  ';': 'I-SEMICOLON'}
normal_label = 'O'
labels_set = list(special_labels.values()) + [normal_label]

In [ ]:
model_args = NERArgs()
model_args.overwrite_output_dir = True
model_args.save_steps = -1
ner_model = NERModel('bert',
                 'bert-base-uncased',
                 labels = labels_set,
                 args=model_args,
                 use_cuda=False)
ner_model.args.max_seq_length = 512
train_df = pd.read_csv('./wikitext/train0-10.csv').dropna()
validation_df = pd.read_csv('./wikitext/validation.csv').dropna()

In [ ]:
train_dataset = ner_model.load_and_cache_examples(train_df[:])
validation_dataset = ner_model.load_and_cache_examples(validation_df[:])

In [ ]:
len(train_dataset), len(validation_dataset)

In [ ]:
from tqdm.auto import tqdm, trange
from torch.optim import Adam, SGD
import torch
from torch.utils.data import DataLoader, TensorDataset

bert_dim = 768
output_dim = len(labels_set)
batch_size = 32
adam_lr = 3e-5
epoch = 1
num_layers = 1
rnn_dim = 512
model = Bertencoder(bert_dim, output_dim, num_layers, rnn_dim)

def train(model, tr_ds, val_ds):
    use_gpu = torch.cuda.is_available()
    device = torch.device('cuda:0' if use_gpu else 'cpu')
    train_dataloader = DataLoader(tr_ds, batch_size=batch_size, shuffle=False)
    validation_dataloader = DataLoader(val_ds, batch_size=batch_size, shuffle=True)
    
    
    if use_gpu:
        model.to(device)
    optimizer = Adam(model.parameters(), lr=adam_lr)
    for i in range(epoch):
        epoch_loss = 0
        batch_iterator = tqdm(train_dataloader, desc=f"Running Epoch {i+1} of {epoch}", mininterval=0)
        
        for (j, batch) in enumerate(batch_iterator):
            batch_x = batch[0].to(device)
            batch_y = batch[3].to(device)
            batch_seg = batch[2].to(device)
            batch_msk = batch[1].to(device)
            loss = model(batch_x, batch_y, batch_seg, batch_msk)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        dev_loss = 0
        predict_idx_label = []

        for (h, val_batch) in enumerate(validation_dataloader):
            val_batch_x = val_batch[0].to(device)
            val_batch_y = val_batch[3].to(device)
            val_batch_seg = val_batch[2].to(device)
            val_batch_msk = val_batch[1].to(device)
            with torch.no_grad():
                loss_val, pre_y = model.test(val_batch_x, val_batch_y, val_batch_seg, val_batch_msk)
            dev_loss += loss_val.item()            
            predict_idx_label.append(pre_y)
        print(f"Epoch {i+1}/{epoch}:\n\tTraining Loss: \t\t{epoch_loss/(j+1):9.4f}\n\tValidation Loss: \t{dev_loss/(h+1):9.4f}")
        print('*******************************************************')


In [ ]:
train(model, train_dataset, validation_dataset)

In [ ]:
print(model)


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from simpletransformers.ner.ner_utils import InputExample

def test_model(to_predict, predictor_model, ner_model):
    predict_examples = [InputExample(i,sentence.split(),[ner_model.args.labels_list[0] for word in sentence.lower().split()],) for i, sentence in enumerate(to_predict)]
    eval_dataset = ner_model.load_and_cache_examples(None, evaluate=True, no_cache=True, to_predict=predict_examples)
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=batch_size)
    
    eval_batch_iterator = tqdm(eval_dataloader, desc=f"Testing!", mininterval=0)
    use_gpu = torch.cuda.is_available()
    device = torch.device('cuda:0' if use_gpu else 'cpu')
    for (j, val_batch) in enumerate(eval_batch_iterator):
        val_batch_x = val_batch[0].to(device)
        val_batch_y = val_batch[3].to(device)
        val_batch_seg = val_batch[2].to(device)
        val_batch_msk = val_batch[1].to(device)
        with torch.no_grad():
            pre_y = predictor_model.predict(val_batch_x, val_batch_seg, val_batch_msk)
        out_label_ids = [[] for _ in range(len(val_batch_x))]

        max_len = np.max([len(x) for x in val_batch_x])
        for index, sentence in enumerate(to_predict):
            for word in sentence.split():
                word_tokens = ner_model.tokenizer.tokenize(word)
                out_label_ids[index].extend([0] + [-100] * (len(word_tokens) - 1))
            out_label_ids[index].insert(0, -100)
            out_label_ids[index].append(-100)
            if len(out_label_ids[index]) < max_len:
                out_label_ids[index].extend([-100] * (max_len - len(out_label_ids[index])))
        xfer_label_ids = np.zeros((len(out_label_ids), max_len))
        for i, out_label_id in enumerate(out_label_ids):
            for j, label in enumerate(out_label_id):
                xfer_label_ids[i][j] = np.int32(label)
        out_label_ids = np.array([list(x) for x in out_label_ids], np.int32)
        preds = pre_y.to("cpu")
        label_map = {i: label for i, label in enumerate(labels_set)}
        out_label_list = [[] for _ in range(out_label_ids.shape[0])]
        preds_list = [[] for _ in range(out_label_ids.shape[0])]
        for i in range(out_label_ids.shape[0]):
            for j in range(out_label_ids.shape[1]):
                if out_label_ids[i, j] != -100:
                    out_label_list[i].append(label_map[out_label_ids[i][j]])
                    preds_list[i].append(label_map[preds[i][j].item()])
    return preds_list

In [ ]:
to_predict = ["First the reading points out that it is imperative for the employees to participate in meetings because meetings are effective for talking about controversial subjects", "This is in direct contrast with the statement of the listening which contends that meetings should be avoided", "He explains that in long meetings bosses spend time reciting introductory information Thus employees do not pay attention in a meeting exceeding an hour Therefore he suggests that the employees politely decline the meeting pointing at a pressing obligation that can be done through a quick phone call or email", "In addition the writer says that it employees should make effort to read all of the training and supplementary material", "When reading such articles the author explains that it is instrumental for the employees to pay attention to the details of the text However the professor refutes this argument claiming that employees should rather try to find the main idea rather than focusing on the intricate details since it takes up an unnecessary amount of time", "He goes as far to claim that employees should refuse to read the entire material even if their bosses request them to do so",
             "Finally according to the reading employees should write their report in one attempt Since the employees are no longer in university they no longer have the luxury to waste time in separating the steps when writing This is challenged by the argument of the listening material that when one does not separate the steps in writing it can be more tire time consuming Thus the professor recommends that the employees write divide the writing into three distinct steps creating an outline writing a first draft and revising the essay In this way the professor ascertains that employees will be able to complete their writing in a timely and accurate manner Finally according to the reading employees should write their report in one attempt Since the employees are no longer in university they no longer have the luxury to waste time in separating the steps when writing This is challenged by the argument of the listening material that when one does not separate the steps in writing it can be more tire time consuming Thus the professor recommends that the employees write divide the writing into three distinct steps creating an outline writing a first draft and revising the essay In this way the professor ascertains that employees will be able to complete their writing in a timely and accurate manner"]
out_labels = test_model(to_predict=to_predict, predictor_model=model, ner_model=ner_model)

for i, sent in enumerate(to_predict):
    print(len(sent.split()))
    print(len(out_labels[i]))
print(out_labels)


In [ ]:
torch.save(model.state_dict(), "./bert_on_blstm_models/0/model.json")
model = Bertencoder(bert_dim, output_dim, num_layers, rnn_dim)
model.load_state_dict(torch.load("./bert_on_blstm_models/0/model.json"))